In [4]:
from citipy import citipy
from pathlib import Path
from pprint import pprint
from time import sleep
import requests as rq
import pandas as pd
import numpy as np
import json

lats = np.random.uniform(-90.000, 90.000, size=2000)
longs = np.random.uniform(-180.000, 180.000, size=2000)
coords = np.stack((lats, longs), axis=1)
resources = Path().cwd().parent / "resources"
# Load key
with open(resources / "keys.json", "r") as keys:
    api_key = json.load(keys)[0]["weather"]

In [5]:
# Get cities
cities = [citipy.nearest_city(c[0], c[1]).city_name for c in coords]
len(cities)

2000

In [13]:
# API calls
def get_cities_data(cities, key):
    url = "https://api.openweathermap.org/data/2.5/weather?"
    for i, city in enumerate(cities):
        query = f"{url}appid={key}&q={city}&units=imperial&lang=en"
        r = rq.get(query).json()
        try:
            name = r["name"]
            country = r["sys"]["country"]
            lat = r["coord"]["lat"]
            lon = r["coord"]["lon"]
            temp_max = r["main"]["temp_max"]
            temp_min = r["main"]["temp_min"]
            humidity = r["main"]["humidity"]
            cloud =  r["clouds"]["all"]
            wind = r["wind"]["speed"]
            desc = r["weather"][0]["description"]
        except:
            print(f"{i}. {city} was not found")
            continue
        yield pd.DataFrame({
                "City":name,
                "Country":country,
                "Lat":lat,
                "Lon":lon,
                "MaxTemp":temp_max,
                "MinTemp":temp_min,
                "Humidity":humidity,
                "Cloudniness":cloud,
                "Wind Speed":wind,
                "Current Description":desc,
            }, index=[i])
# new df
results_df = pd.DataFrame(columns=[
    "City", 
    "Country", 
    "Lat", 
    "Lon", 
    "MaxTemp",
    "MinTemp",
    "Humidity",
    "Cloudniness",
    "Wind Speed",
    "Current Description"
])
# Slowly concatenate the results
# Alternatively, unpack df and use sqlite/other db.
# Function is iterable!
for df in get_cities_data(cities, api_key):
    results_df = pd.concat([results_df, df])
    print(df)

Empty DataFrame
Columns: [City, Country, Lat, Lon, MaxTemp, MinTemp, Humidity, Cloudniness, Wind Speed, Current Description]
Index: []
      City Country   Lat   Lon  MaxTemp  MinTemp  Humidity  Cloudniness  \
0  Ushuaia      AR -54.8 -68.3    33.46    33.46        93           40   

   Wind Speed Current Description  
0        4.61    scattered clouds  
        City Country     Lat      Lon  MaxTemp  MinTemp  Humidity  \
1  Jamestown      US  42.097 -79.2353    71.19    65.95        66   

   Cloudniness  Wind Speed Current Description  
1            0        6.91           clear sky  
     City Country  Lat       Lon  MaxTemp  MinTemp  Humidity  Cloudniness  \
2  Atuona      PF -9.8 -139.0333    76.93    76.93        76           18   

   Wind Speed Current Description  
2       23.67          few clouds  
     City Country      Lat       Lon  MaxTemp  MinTemp  Humidity  Cloudniness  \
3  Hobart      AU -42.8794  147.3294    53.26    49.86        76           20   

   Wind Speed C

In [14]:
# Save to csv 
results_df.to_csv("WeatherPy_Database.csv")

In [15]:
results_df.head()

,City,Country,Lat,Lon,MaxTemp,MinTemp,Humidity,Cloudniness,Wind Speed,Current Description
0,Ushuaia,AR,-54.8000,-68.3000,33.46,33.46,93,40,4.61,scattered clouds
1,Jamestown,US,42.0970,-79.2353,71.19,65.95,66,0,6.91,clear sky
2,Atuona,PF,-9.8000,-139.0333,76.93,76.93,76,18,23.67,few clouds
3,Hobart,AU,-42.8794,147.3294,53.26,49.86,76,20,4.61,few clouds
4,Chokurdakh,RU,70.6333,147.9167,53.47,53.47,56,73,7.56,broken clouds
